In [2]:
import numpy as np
import pandas as pd

In [3]:
input_file = "20210311_v0.2-FSI" # do not put file extension

df = pd.read_csv("{}.csv".format(input_file))

df.describe()

,Entity Registration Date,Entity Profile UEN,Entity Name,Primary Section Description
count,6613,6613,6613,6613
unique,346,6413,6606,1
top,2020-03-13,202031567E,TARGET MULTI ASSET FUND VCC,FINANCIAL AND INSURANCE ACTIVITIES
freq,62,3,2,6613


In [4]:
df

,Entity Registration Date,Entity Profile UEN,Entity Name,Primary Section Description
0,2020-01-09,53408118E,ST HOMES,FINANCIAL AND INSURANCE ACTIVITIES
1,2020-01-13,53408219W,AGNES OH ORGANISATION,FINANCIAL AND INSURANCE ACTIVITIES
2,2020-01-13,53408244X,ONE SG HOLDING,FINANCIAL AND INSURANCE ACTIVITIES
3,2020-01-14,53408260B,CARMINE MOON,FINANCIAL AND INSURANCE ACTIVITIES
4,2020-01-16,53408386B,1ST LG MONEY CHANGER,FINANCIAL AND INSURANCE ACTIVITIES
...,...,...,...,...
6608,2020-12-17,T20VC0183A,SEAVI ADVENT EQUITY VII FUND VCC,FINANCIAL AND INSURANCE ACTIVITIES
6609,2020-12-23,T20VC0185D,WELLINGTON MANAGEMENT FUNDS (SINGAPORE) VCC,FINANCIAL AND INSURANCE ACTIVITIES
6610,2020-12-23,T20VC0187G,PENCO CAPITAL VCC,FINANCIAL AND INSURANCE ACTIVITIES
6611,2020-12-23,T20VC0190G,RAINMAKING VENTURES (S) VCC,FINANCIAL AND INSURANCE ACTIVITIES


In [15]:
import requests
import bs4
import re
from urllib.parse import urlparse, parse_qs

url = "https://www.google.com/search?q={}"
output_file = "{}-result.csv".format(input_file)
size = len(df) # size of df
for i in range(4, 5):
    entity = df.loc[i] # current row record
    entity_name = entity[2] # entity name
    query = "{} singapore website".format(entity_name)
    
    print("Searching for id={} \"{}\"".format(i, query))
    
    # prepare to search "<entity name> singapore website" for more localised search context
    req = requests.get(url.format(query))
    soup = bs4.BeautifulSoup(req.text, "html.parser")
    headers = soup.find_all("h3")
#     links = soup.find_all("div", class_="kCrYT") # html class for google search

#     print(soup)
    links = soup.find_all(href=re.compile(r'\/url\?q=')) # pick top 10 search results & its link
    
    links_extracted = set()
    
    # file write here
    
    f = open(output_file, "a")

    
    for i in range(10):
        google_url = links[i].get('href')
        url = parse_qs(google_url)
        links_extracted.add(url['/url?q'][0])
    
#         print(links_extracted)
        # write out dataframe record, then APPEND links_extracted
#         f.write("Now the file has more content!")
    
    f.close()

        
        
        # "/url?q=https://www.sgpbusiness.com/company/1st-Lg-Money-Changer&sa=U&ved=2ahUKEwjpspb0la3vAhWiJzQIHY-7D-UQFjADegQIChAB&usg=AOvVaw3-P4IBA7DZtL3OsdUAHOD5"
    
        # next, parse to grab URL parameter
    


Searching for id=4 "1ST LG MONEY CHANGER singapore website"


ConnectionError: HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: /search?q=1ST%20LG%20MONEY%20CHANGER%20singapore%20website (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f88b83ee550>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))